# Decision Trees for Classification

In this notebook, we learn how decision trees can solve classification problems and how they are constructed.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.tree import export_graphviz

In [ ]:
# To install graphviz run: 
# ==> conda install python-graphviz
from graphviz import Source

## Decision Tree Classifier

So far, we have only used decision trees in the context of regression tasks. We will now learn how to apply decision trees to classification tasks. Classification is the process of finding a model (e.g., a decision tree) that is capable of dividing a dataset into different classes.

Scikit-Learn provides the class `DecisionTreeClassifier` for building decision trees that can solve classification tasks.

### Load sample dataset

In order to study how a decision tree classifier works, we need a sample dataset. We use the Iris Flower (dt. "Schwertlilie") dataset for this purpose.

The dataset consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters.

Our goal is to predict the species based on the sample features.

In [ ]:
iris_db = load_iris(as_frame=True)
iris_df = iris_db.frame

In [ ]:
# TODO: Inspect the features of the dataset

**According to the description and the `target_names` attribute, the class labels represent the following species:**

- Setosa
- Versicolour
- Virginica

### Train a simple decision tree classifier

In [ ]:
x = iris_df[['petal length (cm)', 'petal width (cm)']].values
y = iris_df['target'].values

In [ ]:
# TODO: Train a decision tree classifier on <x, y>
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
# Question: Check the documentation. 
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# What error function does the decision tree classifier optimize?

### Visualize the decision tree

In [ ]:
export_graphviz(
    model,
    out_file='iris_tree.dot',
    feature_names=["petal length (cm)", "petal width (cm)"],
    class_names=iris_db.target_names,
    rounded=True,
    filled=True
)

In [ ]:
Source.from_file('iris_tree.dot')

In [ ]:
# Question: 
# How does the decision tree look like?
# What are the thresholds?
# How many items per node?

### Visualization of the prediction boundaries

In [ ]:
from matplotlib.colors import ListedColormap
custom_cmap = ListedColormap(['#fafab0', '#9898ff', '#a0faa0'])

plt.figure(figsize=(8, 4))

# Obtain testing samples by taking point on a regular-spaced grid 
lengths, widths = np.meshgrid(np.linspace(0, 7.2, 100), 
                              np.linspace(0, 3, 100))

# ravel() returns a flatten array. Unlike flatten(), it does not return a copy 
# of the original array
x_all = np.stack((lengths.ravel(), widths.ravel()), axis=1)

# Predict the class of each sample point
y_pred = model.predict(x_all).reshape(lengths.shape)

# Create a contour plot using the regular-spaced grid (+ the predict class)
plt.contourf(lengths, widths, y_pred, alpha=0.3, cmap=custom_cmap)

# Visualize the training samples (+ their class value)
for idx, (name, style) in enumerate(zip(iris_db.target_names, ("yo", "bs", "g^"))):
    plt.plot(x[:, 0][y == idx], x[:, 1][y == idx],
             style, label=f"{name}")

plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.axis([0, 7.2, 0, 3])
plt.legend()
plt.show()

### How to construct a decision tree for classification?

We have already introduced the **Classification And Regression Tree (CART)** algorithm when we learned how to construct a decision tree for regression. The construction of decision trees for a classification problem works in an analogous way. In other words, we also aim to split the dataset into two subsets using a single feature $k$ and a threshold $t_k$. These features should be chosen such that the weighted cost $C(k, t_k)$ is minimal.

$C(k, t_k) = \frac{n_{left}}{n} C_{left} + \frac{n_{right}}{n} C_{right}$

where $C_{left}$ and $C_{right}$ is some cost function computed on the left and right subset, $n_{left}$ and $n_{right}$ denotes the number of samples contained in the left and right subset, and $n$ denotes the total number of samples.


However, since we are dealing with a classification problem (which implies that our predicted value is categorical), we cannot use the MSE or MAE as a cost function. Instead, we need a metric that is able to deal with categorical values such as the **Gini Index** or **Entropy**.

#### Gini impurity index

The **Gini impurity index is a measure of diversity** in a dataset. In other words, if we have a set in which all the elements are similar, this set has a low Gini index, and if all the elements are different, it has a large Gini index. 

**Definition:**


In a set with $m$ elements and $n$ classes, with $a_i$ elements belonging to the $i$-th class, the Gini impurity index is

$Gini = 1 - p_1^2 - p_2^2 - ... - p_n^2$ 

where $p_i=\frac{a_i}{m}$. 

This can be interpreted as the probability that if we pick two random elements out of the set, they belong to different classes.

**Note:** The Gini impurity index should not be confused with the Gini coefficient. The Gini coefficient is used in statistics to calculate the income or wealth inequality in countries.

**The following code illustrates how to compute the Gini impurity index:**

In [2]:
def gini_impurity(y):
    
    pass

    # TODO: Calculate the gini impurity

In [ ]:
min_weighted_gini = np.inf
split_threshold = -1
split_feature_idx = -1

# Since each sample is described by two input features, we need to look for the
# best possible split in each dimension
for feature_idx in range(0, 2):
    
    x_feat = x[:, feature_idx]
    
    sort_idx = np.argsort(x_feat, axis=0)
    x_sorted = np.take_along_axis(x_feat, sort_idx, axis=0)
    y_sorted = np.take_along_axis(y, sort_idx, axis=0)
    
    for i in range(1, y.shape[0]):
        
        sort_idx = np.argsort(x, axis=0)

        # Get samples in left and right set
        y_left = y_sorted[:i]
        y_right = y_sorted[i:]

        # Number of samples in each set
        num_left = len(y_left)
        num_right = len(y_right)
        num_total = num_left + num_right

        # Compute GINI impurity for each set
        gini_left = gini_impurity(y_left)
        gini_right = gini_impurity(y_right)

        # Compute the weight sum of both gini impurities
        weighted_gini = num_left / num_total * gini_left + num_right / num_total * gini_right

        if weighted_gini < min_weighted_gini:
            min_weighted_gini = weighted_gini
            split_threshold = (x_sorted[i-1] + x_sorted[i]) / 2
            split_feature_idx = feature_idx

# Obtain the samples in the left and right child node
y_left = y[x[:, split_feature_idx] <= split_threshold]
y_right = y[x[:, split_feature_idx] > split_threshold]

print(f'Threshold: {split_threshold:.3f}')
print(f'Split Feature Idx: {split_feature_idx}')
print(f'Gini Impurity Root: {gini_impurity(y)}')
print(f'Gini Impurity Left: {gini_impurity(y_left)}')
print(f'Gini Impurity Right: {gini_impurity(y_right)}')

#### Entropy

Another measure of homogeneity in a set is **entropy**. It is is based on the physical concept of entropy and is highly important in probability and information theory.

**Definition:**
    
    
In a set with $m$ elements and $n$ classes, with $a_i$ elements belonging to the $i$-th class, the entropy is

$Entropy = -p_1 log_2(p_1) - p_2 log_2(p_2) – ... – p_n log_2(p_n)$

where $p_i = \frac{a_i}{m}$.

#### Entropy vs. Gini impurity

So should you use Gini impurity or entropy? The truth is, most of the time it does not make a big difference: they lead to similar trees. Gini impurity is slightly faster to compute, so it is a good default. However, when they differ, Gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees. See also https://sebastianraschka.com/faq/docs/decision-tree-binary.html

### What if we have categorical features?

So far, in all our decision tree examples, we have only worked with continuous input features. This raises the question of whether we can also process categorical input features. Yes, this is indeed possible, and we can even process non-binary categorical input features if we apply a simple trick.

**Binary categorical features:** Let's first assume we are given a dataset where each sample has a feature saying whether it's a cat or a dog. Since it's a binary feature, we assign each label a numeric class label that is either 0 or 1 depending on the label. When training a decision tree, the numeric labels are simply considered as continuous values. Consequently, the decision threshold that splits the samples into two subsets is 0.5. 

**Non-Binary categorical features:** Let's now assume we are given a dataset where each sample has a feature saying whether it's a cat, dog, or Bird. Obviously, this feature is not binary as there are three possible labels. However, we can convert the feature into several binary features using so-called **one-hot encoding**. In our example, one-hot encoding the feature would result in three distinct binary features encoding the "questions":
- Is the animal a dog?
- Is the animal a cat?
- Is the animal a bird?

Note that Scikit's provides the class `OneHotEncoder` which is able to produce a one-hot encoding of a given feature. 

In [3]:
# TODO: Demonstrate how the OneHotEncoder works

## Instability

We have seen that decision trees are simple to understand and interpret, easy to use, versatile, and powerful.
However they do have a few limitations. First, as you may have noticed, Decision Trees love orthogonal decision boundaries (all splits are perpendicular to an axis), which makes them sensitive to training set rotation. For example, consider a simple linearly separable dataset: on the left, a Decision Tree can split it easily, while
on the right, after the dataset is rotated by 45°, the decision boundary looks unnecessarily convoluted. Although both Decision Trees fit the training set perfectly, it is very likely that the model on the right will not generalize well.

More generally, the main issue with Decision Trees is that they are very sensitive to small variations in the training data.

In [ ]:
# Create a linearily separable dataset
np.random.seed(6)

x = np.random.rand(100, 2) - 0.5
y = (x[:, 0] > 0).astype(np.int64)

In [ ]:
# Rotate the dataset

angle = np.pi / 4  # 45 degrees
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                            [np.sin(angle), np.cos(angle)]])
x_rot = x.dot(rotation_matrix)

In [ ]:
def plot_decision_boundary(clf, X, y, axes, cmap):
    
    x1, x2 = np.meshgrid(np.linspace(axes[0], axes[1], 100),
                         np.linspace(axes[2], axes[3], 100))
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=cmap)
    plt.contour(x1, x2, y_pred, cmap="Greys", alpha=0.8)
    colors = {"Wistia": ["#78785c", "#c47b27"], "Pastel1": ["red", "blue"]}
    markers = ("o", "^")
    
    for idx in (0, 1):
        plt.plot(X[:, 0][y == idx], X[:, 1][y == idx],
                 color=colors[cmap][idx], marker=markers[idx], linestyle="none")
    plt.axis(axes)
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$", rotation=0)


In [ ]:
# Train a classifier to separate the two classes
model = DecisionTreeClassifier(random_state=42)
rot_model = DecisionTreeClassifier(random_state=42)

model.fit(x, y)
rot_model.fit(x_rot, y)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

# plt.sca(): Set the current Axes to ax and the current Figure to the parent of ax.
plt.sca(axes[0])
plot_decision_boundary(model, x, y,
                       axes=[-0.7, 0.7, -0.7, 0.7], cmap="Pastel1")

plt.sca(axes[1])
plot_decision_boundary(rot_model, x_rot, y,
                       axes=[-0.7, 0.7, -0.7, 0.7], cmap="Pastel1")
plt.ylabel("")

plt.show()